<center>
<img src="./pict/folium_logo.png">
<br />
<br />

In [52]:
#!pip install folium # установка Folium, или так: !conda install folium -c conda-forge

In [53]:
# arcgis, pysal, geopandas 

In [105]:
import folium
import geopy # объединенное API для популярных гео-кодирующих сервисов
from geopy.geocoders import Yandex

# https://developers.google.com/maps/documentation/geocoding/start - получить свой API Key
geolocator_google = Yandex(api_key='4f6eaf38-d399-4ad6-af4f-a1974ac4f27d')

In [106]:
# https://mash.ru/letter/coronavirus-2/ - данные взяты отсюда

import pandas as pd

coronavirus_data = pd.read_excel('./data/coronavirus_cases.xlsx')
coronavirus_data.head()

,#,address,date
0,1,"3-й квартал Капотни, 3",28-30 марта
1,2,"4-й квартал Капотни, 9",28-30 марта
2,3,"Авиаторов,6, к1",28-30 марта
3,4,"Адмирала Лазарева, 38",28-30 марта
4,5,"Академика Комарова, 7а",28-30 марта


In [107]:
#чтобы перевести адреса в геовид, необходима gis

from tqdm import tqdm_notebook

lat = []
long = []
strange_address = []

for i in tqdm_notebook(range(coronavirus_data.shape[0])):
    
    address = 'Москва, ' + coronavirus_data.address.iloc[i] #все адреса по Москве
    try:
        location = geolocator_google.geocode(address) # геокодирование -- перевод строки адресо в широту и долготу (возвращает объект локация)
        lat.append(location.latitude) #достаём широту 
        long.append(location.longitude)#достаём долготу
        
    except:
        strange_address.append(address) #должен быть пустой список
        lat.append('NaN')
        long.append('NaN')
        
coronavirus_data['lat'] = lat #сделаем стоблец широт
coronavirus_data['long'] = long # сделаем столбец долгот

In [101]:
strange_address #должен быть пустой список

[]

In [102]:
coronavirus_data.head()

,#,address,date,lat,long
0,1,"3-й квартал Капотни, 3",28-30 марта,55.636712,37.800613
1,2,"4-й квартал Капотни, 9",28-30 марта,55.636341,37.792582
2,3,"Авиаторов,6, к1",28-30 марта,55.648093,37.394027
3,4,"Адмирала Лазарева, 38",28-30 марта,55.545375,37.523824
4,5,"Академика Комарова, 7а",28-30 марта,55.828930,37.592905


In [104]:
geolocator_google.geocode('Московский физико-технический институт')

GeocoderTimedOut: Service timed out

In [110]:
a = geolocator_google.geocode('Москва, Кремль')

In [111]:
??a #узнать про объект

In [90]:
from folium.features import DivIcon #folium - библиотека для визуализации на javascript
#по умолчанию подложкка open street map, но можно поменять
m = folium.Map(location=[55.755826, 37.6172999], # Москва -- задаём координаты
               zoom_start=10) #и увеличение

folium.Marker([55.9297243, 37.5199434], #делаем маркер: координаты
              tooltip='МФТИ', #что будет отображаться если навести курсор на иконку
              icon=folium.Icon(color='green', icon=None)).add_to(m) #set иконку -- None - домик по умолчанию
#т.е. создание объекта и добавление его на крту аттрибутом add_to(m)

folium.Marker([55.752023, 37.6174994], 
              tooltip='Кремль',
              icon=folium.Icon(color='red', icon='star')).add_to(m) #можно создавать кастомные иконки

folium.CircleMarker(location = [55.9297243, 37.5199434], #нарисовать кружочек
                    radius = 50).add_to(m)#радиус в пикселях! при приближении относительный размер 

folium.PolyLine(locations = [(55.929724, 37.5199434), (55.752023, 37.6174994)], #откуда до куда нарис линию (можно ломаную -- сделать больше точек)
                line_opacity = 0.5).add_to(m)

folium.map.Marker(
    [55.755826, 37.6172999], 
    icon=DivIcon( #симпортировали class DivIcon,
        icon_size=(150,36), 
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:red">Какой-нибудь </div>',
        ) #в html формате написали текст, размер, цвет, сам текст
    ).add_to(m)


m

In [108]:
coronavirus_data.date.unique()

array(['28-30 марта', '25 марта-27 марта', '24 марта'], dtype=object)

In [109]:
colors = dict(zip(coronavirus_data.date.unique(), ['green', 'darkred', 'blue']))#сделаем массив цветов по датам в таблице

In [93]:
m = folium.Map(location=[55.755826, 37.6172999], # Москва
               zoom_start=10)

folium.Marker([55.9297243, 37.5199434], #маркер МФТИ
              tooltip='МФТИ',
              icon=folium.Icon(color='green', icon=None)).add_to(m)

folium.Marker([55.752023, 37.6174994], #маркер Кремль
              tooltip='Кремль',
              icon=folium.Icon(color='red', icon='star')).add_to(m)

for i, row in coronavirus_data.iterrows(): #проходим в цикле по строкам таблицы и ставим маркеры соотв датам цветов
    
    color = colors[row.date]
    folium.Marker([row.lat, row.long], 
                  tooltip=row.date, #при наведении курсора отображает нужную дату
                  icon=folium.Icon(color=color, icon='None'),
                  popup=folium.Popup('<b> Адрес: </b>' + '<br>' +   #что будет отображаться, когда нажмём на маркер (адрес)
                                     row.address,                   #с тиле html -- жирнас строка адрес, след - обычная с самим адресом
                                     max_width=500)).add_to(m)

m #объект m (map)

In [94]:
m #можем запустить его с любой ячейки

In [ ]:
m.save('./data/coronavirus_cases_map.html') # конвертирование в html (сохранить)

In [95]:
from folium.plugins import HeatMap #объект тепловая карта

coronavirus_heatmap = coronavirus_data[['lat', 'long']].copy()
coronavirus_heatmap['weight'] = 1
coronavirus_heatmap = coronavirus_heatmap.values.tolist()

m = folium.Map(location=[55.755826, 37.6172999], tiles='stamentoner', #изменили формат подложки -- не openmaps, a stamentoner
               zoom_start=10)

HeatMap(coronavirus_heatmap).add_to(m)#на вход передаём лист -- пара координат и вес точки (> вес -> ярче)
#см подробнее аргументы, можно регулировать радиусы схлопывания кружков и др недефолтные вещи

m

In [96]:
from folium.plugins import HeatMapWithTime #изменения тепл карты во времени

coronavirus_heatmap = coronavirus_data[['date', 'lat', 'long']].copy()
coronavirus_heatmap['weight'] = 1

coronavirus_heatmaptime = []
for dates in [['24 марта'], 
              ['24 марта', '25 марта-27 марта'], 
              ['24 марта', '25 марта-27 марта', '28-30 марта']]:
    coronavirus_heatmaptime.append(coronavirus_heatmap[coronavirus_heatmap.date.isin(dates)][['lat', 
                                                                         'long', 'weight']].values.tolist())
#список списков по датам    
m = folium.Map(location=[55.755826, 37.6172999], 
               zoom_start=9)

HeatMapWithTime(coronavirus_heatmaptime).add_to(m)#объект -- список списков

m

In [97]:
geolocator_google.geocode('Москва, Троицк')

Location(Троицк, Москва, Россия, (55.484842, 37.305561, 0.0))

In [98]:
geolocator_google.geocode('Тверская область, Тверь')

Location(Тверь, Россия, (56.859847, 35.911995, 0.0))

In [99]:
# Как найти полигоны?

# Заходим сюда: https://nominatim.openstreetmap.org/search.php?q=&polygon_geojson=1&viewbox=
# Ищем необходимый объект
# В описании объекта находим код OSM 
# Заходим сюда: http://polygons.openstreetmap.fr/index.py 
# Вбиваем код OSM
# Получаем GeoJSON 
# Profit! 


#большинство полигонов есть на сайте, но если есть, то плохо, и пришлось бы создавать их, самостоятельно вбивая координаты

In [100]:
import json #объекты-полигоны (последовательность точек, которые нужно отрисовывать)

with open('./data/moscow.json') as json_file:
    moscow_polygon = json.load(json_file) #создадим объект с полигоном Москвы
    
with open('./data/new_moscow.json') as json_file:
    new_moscow_polygon = json.load(json_file) #Москвы и новой Москвы
    
with open('./data/tver_region.json') as json_file:
    tver_region_polygon = json.load(json_file) #Тверская область
    
m = folium.Map(location=[55.755826, 37.6172999], # Москва
               zoom_start=7) #создаём карту

m.choropleth(new_moscow_polygon, fill_color='yellow') #добавляем к карте полигоны
m.choropleth(moscow_polygon, fill_color='brown')
m.choropleth(tver_region_polygon, fill_color='blue')

#далее маркеры-подписи полигонов
folium.map.Marker(
    [55.755826, 37.6172999],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:green"> Москва </div>',
        )
    ).add_to(m)

folium.map.Marker(
    [55.4903205, 37.3032108], # Троицк
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:red"> Троицк </div>',
        )
    ).add_to(m)


folium.map.Marker(
    [56.85872140000001, 35.9175965], # Тверь
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:white"> Тверь </div>',
        )
    ).add_to(m)

m

C:\Users\Admin\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
